# Librerias

In [ ]:
import sys
sys.path.append('..')

from modulodirectorio import*
from Utils import*

import datetime
import os
import zipfile
import glob
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import numpy as np
import seaborn as sns
import pandas as pd
import re
import openpyxl
import shutil #Mover ficheros
import holidays
import json

In [ ]:
# AQUI SE CREA EL DICCIONARIO PARA CREAR LOS RANGOS DE NUNMERACION PARA CATEGORIZAR EL PROVEEDOR ORIGEN CUANDO EL TIPO DE MOVIMIENTO ES INSERTAR

rangos_numeracion = pd.read_csv(get_path('movimientos_portabilidad'),  delimiter=',')

dict_rango = {}


for index, row in  rangos_numeracion.iterrows():

    dict_rango[row['NOMBRE OPERADOR ']] = row['RANGO DE NUMERACIÓN 1']

dict_rango = {clave: valor.split(',') for clave, valor in dict_rango.items()}
dict_rango  = {clave: [subvalor.split('-') for subvalor in valor] for clave, valor in dict_rango.items()}


# Proceso

In [ ]:
base_dir_mov = get_path('input_control_ficheros')
mapeo_competidores = pd.read_excel(get_path('mapeo_proveedor'))

#------------------------------------------------------------------------------------------------------------------------

movimientos_portabilidad = pd.read_csv(get_path('nrn_portabilidad'),  delimiter=';', encoding='ansi')

movimientos_portabilidad = movimientos_portabilidad[~movimientos_portabilidad['NOMBRE OPERADOR '].isnull()]

dict_operador = {}


for index, row in  movimientos_portabilidad.iterrows():

    dict_operador[row['NOMBRE OPERADOR ']] = row['nrn']

dict_operadorcoma = {clave: valor.split(',') for clave, valor in dict_operador.items()}
dict_operadorguion  = {clave: [subvalor.split('-') for subvalor in valor] for clave, valor in dict_operadorcoma.items()}
dict_operadorguion['BAJA DEFINITIVA'] = [[-1]]

#------------------------------------------------------------------------------------------------------------------------
files = []

#Es un fichero .dat

for nameCSV in glob.glob(os.path.join(base_dir_mov ,"*.dat")):
    name=str(nameCSV)
    name= pd.read_csv(name, delimiter=';', header = 0, skiprows = 1, names =['TIPO MOVIMIENTO', 'MSISDN', 'ORIGEN','DESTINO','FECHA','CODIGO'])
    files.append(name)
    
movimientos = pd.concat(files)


movimientos[['FECHA', 'HORA']] = movimientos['FECHA'].str.split("/", expand=True)
movimientos['ORIGEN'] = movimientos['ORIGEN'].fillna(-1).astype(int)
movimientos['DESTINO'] = movimientos['DESTINO'].fillna(-1).astype(int)
movimientos['FECHA'] = pd.to_datetime(movimientos['FECHA'], format='%Y%m%d')
movimientos['FECHA'] = pd.to_datetime(movimientos['FECHA'], format='%d/%m/%Y')
movimientos['FECHA'] = movimientos['FECHA'].dt.strftime('%d/%m/%Y')
movimientos['FECHA'] = pd.to_datetime(movimientos['FECHA'], format='%d/%m/%Y')
movimientos['ORIGEN'] = movimientos['ORIGEN'].fillna(-1).astype(int)
movimientos['DESTINO'] = movimientos['DESTINO'].fillna(-1).astype(int)
movimientos['PERIODO'] = movimientos['FECHA'].dt.to_period('M')



for categoria, valores in dict_operadorguion.items():
    
    for elemento in valores:

        if isinstance(elemento, list):  # Verificar si es una lista
            
            try:
                valor_minimo, valor_maximo = elemento
                # Utilizar `between` para verificar si la columna está dentro del rango de valores
                mask = movimientos['ORIGEN'].between(int(valor_minimo), int(valor_maximo))
                movimientos.loc[mask, 'PROVEEDOR ORIGEN'] = categoria
            
            except:

                for valor in elemento:
                    mask = movimientos['ORIGEN'] == int(valor)
                    movimientos.loc[mask, 'PROVEEDOR ORIGEN'] = categoria

        else:  # Verificar si es una lista
            for valor in elemento:
                # Utilizar operador de igualdad `==` para comparar con cada valor de la lista
                mask = movimientos['ORIGEN'] == int(valor)
                movimientos.loc[mask, 'PROVEEDOR ORIGEN'] = categoria




# Verificar si el NRN coincide con alguna categoría del diccionario

for categoria, valores in dict_operadorguion.items():
    
    for elemento in valores:

        if isinstance(elemento, list):  # Verificar si es una lista
            
            try:
                valor_minimo, valor_maximo = elemento
                # Utilizar `between` para verificar si la columna está dentro del rango de valores
                mask = movimientos['DESTINO'].between(int(valor_minimo), int(valor_maximo))
                movimientos.loc[mask, 'PROVEEDOR DESTINO'] = categoria
            
            except:

                for valor in elemento:
                    mask = movimientos['DESTINO'] == int(valor)
                    movimientos.loc[mask, 'PROVEEDOR DESTINO'] = categoria

        else:  # Verificar si es una lista
            for valor in elemento:
                # Utilizar operador de igualdad `==` para comparar con cada valor de la lista
                mask = movimientos['DESTINO'] == int(valor)
                movimientos.loc[mask, 'PROVEEDOR DESTINO'] = categoria




movimientos['MSISDN'] = movimientos['MSISDN'].astype(str)

movimientos = movimientos.sort_values('FECHA', ascending=False)

movimientos.loc[movimientos['ORIGEN'] == -1, 'PROVEEDOR ORIGEN'] = ''

movimientos.loc[movimientos['ORIGEN'] == -1, 'ORIGEN'] = ''

movimientos.loc[movimientos['DESTINO'] == -1, 'DESTINO'] = ''

movimientos['ACCION'] = movimientos.apply(cat_accion, axis=1)


In [ ]:
# Divido el dataframe en dos partes para tratarlos de manera independiente y posteriormente concatenarlos.

mov_ins = movimientos[movimientos['TIPO MOVIMIENTO'] == 'INSERTAR']
movimientos = movimientos[movimientos['TIPO MOVIMIENTO'] != 'INSERTAR']

# Para poder iterar y comparar necesito que todas las variables tengan formato integer
mov_ins['MSISDN'] = mov_ins['MSISDN'].astype(int)


for categoria, valores in dict_rango.items():
    
    for elemento in valores:

        if isinstance(elemento, list):  # Verificar si es una lista
            
            try:
                
                valor_minimo, valor_maximo = elemento
                # Utilizar `between` para verificar si la columna está dentro del rango de valores
                mask = mov_ins['MSISDN'].between(int(valor_minimo), int(valor_maximo))
                mov_ins.loc[mask, 'PROVEEDOR ORIGEN'] = categoria
            
            except:
                for valor in elemento:
                    mask = mov_ins['MSISDN'] == int(valor)
                    mov_ins.loc[mask, 'PROVEEDOR ORIGEN'] = categoria

        else:  # Verificar si es una lista
            for valor in elemento:
                # Utilizar operador de igualdad `==` para comparar con cada valor de la lista
                mask = mov_ins['MSISDN'] == int(valor)
                mov_ins.loc[mask, 'PROVEEDOR ORIGEN'] = categoria

mov_ins['MSISDN'] = mov_ins['MSISDN'].astype(str)

movimientos = pd.concat([movimientos,mov_ins], ignore_index=True)

movimientos = movimientos.merge(mapeo_competidores, left_on ='PROVEEDOR DESTINO', right_on='Proveedor_2', how = 'left')

movimientos = movimientos.rename(columns={"Proveedor_2": "Proveedor Destino nivel 1"})

mov_sin_duplicados_mes = movimientos.drop_duplicates(keep = 'first', subset=['MSISDN', 'PERIODO'])

In [ ]:
movimientos.to_csv(get_path('historico_movimientos_portabilidad') / 'Movimientos_portabilidad_procesado_1.csv', sep = ',', index = False)
mov_sin_duplicados_mes.to_csv(get_path('historico_movimientos_portabilidad') / 'Movimientos_portabilidad_procesado_2.csv', sep = ',', index = False )